# 前処理

## インポート

In [ ]:
from fabric import Connection, Config

## 接続情報
IPアドレスはDefault Switchで割り当てられたDHCPアドレスを取得

In [ ]:
remote_host = '192.168.128.10'
remote_user = 'vagrant'
key_file    = '.vagrant/machines/slurm-master/hyperv/private_key'

## fabric設定

In [ ]:
config = Config(overrides={
    'run': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': False,'hide': False,'timeout': 20},
    'sudo': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': True,'hide': False,'timeout': 20,'password': None},
    'timeouts': {'connect': 5,'command': 30,},
})

# 接続

## SSH接続

In [ ]:
c = Connection(
    port=22,
    host=remote_host,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

In [ ]:
c.run('sinfo --version')

# 処理実行

In [ ]:
for node in ["slurm-master", "slurm-node1", "slurm-node2"]:
    c.run(f"ssh -o StrictHostKeyChecking=no vagrant@{node} 'hostname'")

In [ ]:
c.run('sinfo')

## 各サービスの再起動
ノードが認識されていないなど問題がある場合は各ノードのサービスを再起動する

In [ ]:
c.run('sudo systemctl restart munge')

In [ ]:
c.run('ssh vagrant@slurm-node1 "sudo systemctl restart munge"')

In [ ]:
c.run('ssh vagrant@slurm-node2 "sudo systemctl restart munge"')

In [ ]:
c.run('sudo systemctl restart slurmctld')

In [ ]:
c.run('sudo systemctl restart slurmd')

In [ ]:
c.run('ssh vagrant@slurm-node1 "sudo systemctl restart slurmd"')

In [ ]:
c.run('ssh vagrant@slurm-node2 "sudo systemctl restart slurmd"')

## テスト

In [ ]:
c.run('sinfo')

In [ ]:
c.run('sinfo -Nl')

In [ ]:
c.run('srun hostname')

In [ ]:
c.sudo('srun -N 3 hostname')

In [ ]:
c.run('sbatch --wrap="hostname"')

In [ ]:
c.run('squeue')

In [ ]:
c.run('ls -l')

In [ ]:
c.run('cat slurm-*.out')

In [ ]:
c.close()